### Streamlit으로 ChatGPT 같은 웹앱 제작

Streamlit은 Python으로 간단하게 웹 앱을 만들 수 있는 오픈소스 프레임워크

**특징**
+ 간편함: HTML, CSS, JS 몰라도 Python 코드만으로 웹 앱 작성 가능

+ 빠른 빌드: 코드 수정 시 앱이 자동으로 업데이트됨 (hot reload)

+ 데이터 중심: pandas, matplotlib, plotly, sklearn 등과 잘 호환됨

+ 인터랙션: 슬라이더, 체크박스, 버튼 등의 UI 요소를 쉽게 추가 가능

**링크**: https://docs.streamlit.io/develop/api-reference

### 1. Streamlit 사용해보기

+ 순서 1. root 경로 > 폴더 생성('00-MyProject') > 'main.py' 생성

+ 순서 2. main.py 파일에 다음 코드 작성

In [ ]:
import streamlit as st

st.title("안녕하세요! 반갑습니다 :)")

+ 순서 3. 'TERMINAL' 창에 다음 명령어 실행
    - cd 00-MyProject
    - streamlit run main.py

![](https://codetutorbot.blob.core.windows.net/image/실행1.png)

### 2. Markdown으로 정보 페이지 구성해보기

+ 링크: https://docs.streamlit.io/develop/api-reference/text/st.markdown

In [ ]:
multi = '''If you end a line with two spaces,
a soft return is used for the next line.

Two (or more) newline characters in a row will result in a hard return.
'''

st.markdown(multi)

### 3. ChatInput

+ 링크: https://docs.streamlit.io/develop/api-reference/chat/st.chat_input

In [ ]:
user_input = st.chat_input("궁금한걸 물어보세요.")
if user_input:
    st.write(f"사용자: {user_input}")

### 4. ChatMessage

+ 링크: https://docs.streamlit.io/develop/api-reference/chat/st.chat_message

In [ ]:
user_input = st.chat_input("궁금한걸 물어보세요.")
if user_input:
    # st.write(f"사용자: {user_input}")
    with st.chat_message("user"):
        st.write(user_input)

In [ ]:
user_input = st.chat_input("궁금한걸 물어보세요.")
if user_input:
    # st.write(f"사용자: {user_input}")
    # with st.chat_message("user"):
    st.chat_message("user").write(user_input)

In [ ]:
user_input = st.chat_input("궁금한걸 물어보세요.")
if user_input:
    st.chat_message("user").write(user_input)
    st.chat_message("assistant").write(user_input)

### 5. session_state

+ 링크: https://docs.streamlit.io/develop/api-reference/caching-and-state/st.session_state

+ Streamlit에서 앱 상태(변수)를 유지하기 위한 저장소.


+ 앱이 retrun될 때도 데이터를 잃지 않고 기억하도록 도와주는 딕셔너리 형태의 객체.

+ 왜 사용해야하는가?

    - Streamlit은 사용자 입력, 버튼 클릭, 파일 업로드 같은 이벤트가 발생할 때마다 전체 코드를 다시 실행합니다.

    - 그런데 일반 변수는 rerun되면 초기화돼버리기 때문에,
사용자 세션마다 값(상태)을 유지하려면 st.session_state를 써야 합니다.

In [ ]:
# 예제 1 : session_state에 해당 key가 존재 하지 않으면 value로 초기화
if 'key' not in st.session_state:
    st.session_state['key'] = 'value'

st.write(st.session_state['key'])

In [ ]:
# 예제 2: 예제 1과 동일함
st.write(st.session_state.key)

In [ ]:
# 예제 3: session_state 모두 출력
if 'key' not in st.session_state:
    st.session_state.key = 'value'

st.write(st.session_state)

In [ ]:
# 예제 4: key가 없는 경우
st.write(st.session_state.value) # st.session_state['value']

### 6. session_state로 대화 정보 관리하기
+ 1. 대화기록을 저장하기 위한 session_state['messages]를 정의
    - 처음 1번만 실행되도록 설정

+ 2. 사용자 입력이 들어올 때마다, 대화 기록을 저장
    - ("user", user_input)
    - ("assistant", user_input)

+ 3. 전체 대화 기록 출력

+ 4. 이전 대화 출력


In [ ]:
import streamlit as st

st.title("안녕하세요! 반갑습니다 :)")

# 1. 처음 1번만 대화기록을 저장하기 위한 용도로 사용
if "messages" not in st.session_state:
    st.session_state['messages'] = []

# 4. 이전 대화 출력
for role, message in st.session_state['messages']:
    st.chat_message(role).write(message)

user_input = st.chat_input("궁금한걸 물어보세요.")

# 현재 대화 출력
if user_input:
    st.chat_message("user").write(user_input)
    st.chat_message("assistant").write(user_input)

    # 2. 대화 기록 저장
    st.session_state["messages"].append(("user", user_input))
    st.session_state["messages"].append(("assistant", user_input))    

# 3. 전체 대화 기록 출력
st.write(st.session_state['messages'])

### 7. LangChain 방식으로 바꾸기

LangChain의 ChatMessage 객체를 제대로 활용하기 위한 완성형 구조로 변경해 봅시다.

+ 저장 형식도 ChatMessage로 통일

+ 출력도 ChatMessage 기반으로 정리

+ 함수로 구조화하여 가독성과 유지보수성 향상`

#### 무엇이 달라졌는가?

✅ 추가 1: ChatMessage import
+ LangChain에서 제공하는 메시지 구조인 `ChatMessage` 클래스를 가져옵니다.

+ 이후 대화 기록을 구조화된 형태(role, content 속성 포함)로 저장하기 위해 사용됩니다.


✅ 추가 2: add_message() 함수 정의
+ 새로운 대화를 ChatMessage 객체로 만들어 `st.session_state["messages"]`에 저장하는 함수입니다.

+ 매번 직접 .append()를 호출하지 않고 함수로 감싸 가독성과 재사용성을 높였습니다.

✅ 추가 3: add_message() 함수 호출
+ 사용자의 입력이 있을 경우, add_message() 함수를 이용해

    - 사용자 메시지 `(role="user")`와

    - 어시스턴트 메시지 `(role="assistant")`를 각각 저장합니다.

✅ 추가 4: print_messages() 함수 정의
+ `st.session_state["messages"]`에 저장된 ChatMessage 객체들을 하나씩 출력해주는 함수입니다.

+ `chat_message.role`과 `chat_message.content`를 사용해 역할과 내용을 꺼내 `Streamlit 채팅 UI`로 출력합니다.

✅ 추가 5: print_messages() 함수 호출
+ Streamlit 앱 실행 시점에 저장된 대화들을 화면에 출력합니다.

+ 과거 대화를 복원하여 사용자에게 보여주는 역할을 합니다.

In [ ]:
import streamlit as st
# 추가 1
from langchain_core.messages.chat import ChatMessage

st.title("안녕하세요! 반갑습니다 :)")

if "messages" not in st.session_state:
    st.session_state['messages'] = []

# for role, message in st.session_state['messages']:
#     st.chat_message(role).write(message)


# 추가 4: 이전 대화를 출력하는 함수 정의
def print_messages():
    for chat_message in st.session_state['messages']:
        st.chat_message(chat_message.role).write(chat_message.content)

# 추가 5: 이전 대화를 출력하는 함수 호출
print_messages()

# 추가 2: 새로운 메시지 추가 함수 정의
def add_message(role, message):
    st.session_state["messages"].append(ChatMessage(role=role, content=message))


user_input = st.chat_input("궁금한걸 물어보세요.")

if user_input:
    st.chat_message("user").write(user_input)
    st.chat_message("assistant").write(user_input)

    # st.session_state["messages"].append(("user", user_input))
    # st.session_state["messages"].append(("assistant", user_input))
    
    # 추가 3: 새로운 메시지 추가 함수 호출
    add_message("user", user_input)
    add_message("assistant", user_input)

# st.write(st.session_state['messages'])

### 8. LCEL 적용하기

LCEL(ChatMessagePrompt)를 적용하여 AI 답변이 표시되게 변경해보자.

✅ 추가 1: langchain과 dotenv 모듈 임포트


✅ 추가 2: creat_chain() 함수 정의
+ 역할: 프롬프트 + LLM + 파서로 구성된 체인을 매번 직접 생성하지 않고 함수로 추출하여 재사용 가능하게 함.
    - 매번 ChatPromptTemplate 등 체인을 새로 만들지 않아도 됨

✅ 변경 1: AI의 답변 처리 방식 변경
+ 사용자의 입력(user_input)을 LangChain 기반 체인에 전달해 AI 응답을 생성하도록 변경.

+ invoke() 함수로 프롬프트 → 모델 → 파서 순으로 처리한 결과를 받아 출력함.


✅ 변경 2: 세션 상태에 AI 응답 저장
+ 변경 후: 대화 기록을 `st.session_state["messages"]`에 저장해서 채팅 히스토리 유지 가능.`

코드 캡쳐 링크 바로가기: https://codetutorbot.blob.core.windows.net/image/실행2.png

### 9. 사이드바 생성
+ 사이드바를 추가해보자.
    - 대화 기록을 초기화하는 버튼이 담겨 있음.
    
![](https://codetutorbot.blob.core.windows.net/image/실행3.png)